## Extracting data

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from functools import reduce
import numpy as np
import time
from bs4 import BeautifulSoup
import requests
import nltk 

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import string
from nltk.tokenize import RegexpTokenizer

In [ ]:
def extractring_page(url):
    try:
        l=[]
        soup=BeautifulSoup(requests.get(url).text, "html.parser")
        time.sleep(2)
        try:
            y=soup.findAll('p',{'titolo text-primary'})
            for house in y:
                s='https://www.immobiliare.it'
                s1=house.contents[1]['href']
                if s1.startswith("https")==False:
                    s1=s+s1
                l.append(s1)
        except:
            pass
    except:
        l=['The page does not exit']
    return(l)

In [ ]:
link=[]
for i in range(600):
    url = 'https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag='+str(i+1)
    link.append(extractring_page(url))
link=reduce(lambda x,y: x+y,link)

In [ ]:
link1=pd.DataFrame()
link1['links']=link
link.to_csv("Scrappinglink.csv")

In [ ]:
l=pd.read_csv("Scrappinglink.csv")
l=list(l['links'])

In [ ]:
def Objective_page(url):
    try:
        tag=BeautifulSoup(requests.get(url).text, "html.parser")
        time.sleep(2)
        l1=['piano','bagni','superficie','locali']
        df=[]
        x=tag.findAll('div',{'class':'im-property__features'})
        y=tag.findAll('div',{'class':'clearfix description'})
        d={}
        house=[text for text in x[0].stripped_strings]
        house.reverse()
        house1=[text for text in y[0].stripped_strings]
        house1=" ".join(house1)
        for i in l1:
            if i in house:
                index=house.index(i)
                if i=='superficie':
                    d[i]=house[index+3]
                else:
                    d[i]=house[index+1]
                if house[-1].find('€')!=-1:
                    d['prezzo(€)']=house[-1]
        d['description']=house1
    except:
        d['description']="there was a problem in this link"
    del tag
    return(d)

In [ ]:
from multiprocessing import Pool
with Pool(5) as p:
    records = p.map(Objective_page, l)
p.terminate()
p.join()

In [ ]:
df1=pd.DataFrame.from_dict(records)

In [ ]:
df2.to_csv("Scraping.csv")

## Cleaning data